# Day 1: Prerequisites & Setup

## Welcome to Qwen-Agent! 🎉

### Today's Learning Objectives:
1. Understand what LLM agents are and why they're powerful
2. Learn the Qwen-Agent architecture at a high level
3. Set up your development environment
4. Configure API access (DashScope or OpenAI-compatible)
5. Run your first agent and understand streaming vs. non-streaming

### Time Required: 1.5-2 hours

---

## Part 1: What Are LLM Agents?

### Traditional LLM Usage:
```
You → Prompt → LLM → Response → You
```

**Limitations:**
- LLM has no access to real-world data
- Cannot perform actions (run code, search web, etc.)
- Limited to knowledge cutoff date
- No memory beyond current conversation

### LLM Agent Approach:
```
You → Agent → LLM ←→ Tools (Code, Search, APIs, etc.)
              ↓
            Response
```

**Capabilities:**
- **Tool Use**: Can execute code, search web, query databases
- **Planning**: Breaks complex tasks into steps
- **Memory**: Maintains context across conversations
- **Reasoning**: ReAct pattern (Reason → Act → Observe → Repeat)

### Why Qwen-Agent?
1. **Built for Qwen models** - Optimized for Qwen3, QwQ, Qwen-VL families
2. **Production-ready** - Powers Qwen Chat (chat.qwen.ai)
3. **Flexible** - Works with any OpenAI-compatible API
4. **Feature-rich** - RAG, multi-agent, GUI included
5. **Well-documented** - Active community and examples

## Part 2: Qwen-Agent Architecture Overview

### Core Components:

```
┌─────────────────────────────────────────┐
│           Qwen-Agent Framework          │
├─────────────────────────────────────────┤
│                                         │
│  ┌──────────┐  ┌──────────┐  ┌──────┐  │
│  │  Agents  │  │   Tools  │  │ LLMs │  │
│  └──────────┘  └──────────┘  └──────┘  │
│       │              │           │      │
│  ┌────▼──────────────▼───────────▼───┐ │
│  │     Message Communication Layer    │ │
│  └────────────────────────────────────┘ │
│                                         │
│  ┌──────────┐  ┌──────────┐            │
│  │  Memory  │  │   GUI    │            │
│  └──────────┘  └──────────┘            │
│                                         │
└─────────────────────────────────────────┘
```

#### 1. **Agents** (High-level orchestration)
   - `BasicAgent` - Simple LLM wrapper
   - `Assistant` - Full-featured agent with tools + RAG
   - `FnCallAgent` - Function calling specialist
   - `GroupChat` - Multi-agent coordinator
   - Custom agents - You build your own!

#### 2. **Tools** (Extend agent capabilities)
   - Built-in: `code_interpreter`, `web_search`, `image_gen`
   - Custom: You define your own tools
   - MCP: Community-built tool servers

#### 3. **LLMs** (Language model backends)
   - DashScope (Qwen models via API)
   - OpenAI-compatible (vLLM, Ollama, OpenAI)
   - Local models (transformers, OpenVINO)

#### 4. **Messages** (Communication protocol)
   - Standardized message format
   - Supports text, images, audio, video
   - Function call representation

#### 5. **Memory** (Context management)
   - Conversation history
   - RAG knowledge base
   - File storage

#### 6. **GUI** (User interface)
   - Gradio-based web UI
   - Rapid prototyping
   - Production deployment

## Part 3: Environment Setup

### Step 1: Check Python Version
Qwen-Agent requires Python 3.8+, but GUI features need 3.10+

In [ ]:
import sys
print(f"Python version: {sys.version}")
print(f"Version info: {sys.version_info}")

# Check if version is sufficient
if sys.version_info >= (3, 10):
    print("✅ Python 3.10+ detected - All features available!")
elif sys.version_info >= (3, 8):
    print("⚠️  Python 3.8-3.9 detected - GUI features not available")
else:
    print("❌ Python version too old - Please upgrade to 3.10+")

### Step 2: Install Qwen-Agent

There are two installation options:

#### Option A: Minimal Installation (Function calling only)
```bash
pip install qwen-agent
```

#### Option B: Full Installation (Recommended for this course)
```bash
pip install -U "qwen-agent[gui,rag,code_interpreter,mcp]"
```

**What each feature provides:**
- `[gui]` - Gradio web interface (requires Python 3.10+)
- `[rag]` - Document processing and retrieval
- `[code_interpreter]` - Python code execution
- `[mcp]` - Model Context Protocol support

Let's check if it's already installed:

In [ ]:
try:
    import qwen_agent
    print(f"✅ Qwen-Agent is installed!")
    print(f"Version: {qwen_agent.__version__ if hasattr(qwen_agent, '__version__') else 'Unknown'}")
    print(f"Location: {qwen_agent.__file__}")
except ImportError:
    print("❌ Qwen-Agent not found. Please run:")
    print('   pip install -U "qwen-agent[gui,rag,code_interpreter,mcp]"')

### Step 3: Verify Key Dependencies

In [ ]:
# Check core dependencies
dependencies = {
    'dashscope': 'DashScope API client',
    'openai': 'OpenAI API client',
    'pydantic': 'Data validation',
    'tiktoken': 'Token counting',
    'json5': 'JSON parsing',
}

optional_dependencies = {
    'gradio': 'Web UI (requires Python 3.10+)',
    'jupyter': 'Code interpreter',
    'pandas': 'Data processing',
}

print("Core Dependencies:")
for package, description in dependencies.items():
    try:
        __import__(package)
        print(f"  ✅ {package:15} - {description}")
    except ImportError:
        print(f"  ❌ {package:15} - {description} (MISSING)")

print("\nOptional Dependencies:")
for package, description in optional_dependencies.items():
    try:
        __import__(package)
        print(f"  ✅ {package:15} - {description}")
    except ImportError:
        print(f"  ⚠️  {package:15} - {description} (optional)")

## Part 4: API Configuration

### Option A: DashScope (Recommended for Qwen models)

DashScope is Alibaba Cloud's model service platform. It provides:
- Official Qwen models (Qwen3, QwQ, Qwen-VL, etc.)
- Pay-as-you-go pricing
- Free tier available
- No local GPU required

**Get your API key:**
1. Visit: https://dashscope.console.aliyun.com/
2. Sign up / Log in
3. Go to API-KEY management
4. Create new key

**Set up your key:**

In [ ]:
import os

# Method 1: Set environment variable (recommended)
# Uncomment and replace with your actual key:
# os.environ['DASHSCOPE_API_KEY'] = 'your-api-key-here'

# Method 2: Load from .env file (even better!)
try:
    from dotenv import load_dotenv
    load_dotenv()  # Loads from .env file in current directory
    print("✅ Loaded .env file")
except ImportError:
    print("⚠️  python-dotenv not installed (optional)")

# Check if API key is set
if os.getenv('DASHSCOPE_API_KEY'):
    key = os.getenv('DASHSCOPE_API_KEY')
    print(f"✅ DashScope API key found: {key[:10]}...{key[-5:]}")
else:
    print("❌ DASHSCOPE_API_KEY not set")
    print("   Set it with: os.environ['DASHSCOPE_API_KEY'] = 'your-key'")
    print("   Or create a .env file with: DASHSCOPE_API_KEY=your-key")

### Option B: OpenAI-Compatible Services

You can also use:
- **OpenAI** - Official API (GPT-4, GPT-3.5, etc.)
- **vLLM** - Self-hosted high-performance inference
- **Ollama** - Local CPU/GPU inference
- **Azure OpenAI** - Microsoft's OpenAI service

Example configurations:

```python
# OpenAI
llm_cfg = {
    'model': 'gpt-4o',
    'model_type': 'oai',
    'api_key': os.getenv('OPENAI_API_KEY')
}

# vLLM (self-hosted)
llm_cfg = {
    'model': 'Qwen2.5-7B-Instruct',
    'model_server': 'http://localhost:8000/v1',
    'api_key': 'EMPTY'
}

# Ollama (local)
llm_cfg = {
    'model': 'qwen2.5:7b',
    'model_server': 'http://localhost:11434/v1',
    'api_key': 'EMPTY'
}
```

For this course, we'll use **DashScope** for simplicity.

## Part 5: Your First Agent - "Hello World"

Let's create the simplest possible agent to verify everything works.

### Understanding the Code:
1. Import the `Assistant` class (a pre-built agent)
2. Configure which LLM to use
3. Create an agent instance
4. Send messages and receive responses

In [ ]:
from qwen_agent.agents import Assistant

# Step 1: Configure the LLM
llm_cfg = {
    'model': 'qwen-max-latest',  # Use the latest Qwen model
    # API key will be read from DASHSCOPE_API_KEY environment variable
}

# Step 2: Create an agent
bot = Assistant(llm=llm_cfg)

# Step 3: Prepare a message
messages = [
    {'role': 'user', 'content': 'Hello! What is 2+2?'}
]

# Step 4: Get response (non-streaming)
print("Running agent...\n")
response = bot.run_nonstream(messages=messages)

# Step 5: Display the response
for msg in response:
    print(f"[{msg['role']}]: {msg.get('content', '')}")

### 🎉 If you see a response above, congratulations!

You've successfully:
- Installed Qwen-Agent
- Configured API access
- Created your first agent
- Sent a message and received a response

## Part 6: Streaming vs. Non-Streaming Responses

### What's the difference?

**Non-Streaming (`run_nonstream`):**
- Waits for complete response
- Returns all messages at once
- Simpler to use
- User waits longer

**Streaming (`run`):**
- Returns response incrementally
- Better user experience (like ChatGPT)
- More complex to handle
- Lower perceived latency

### Visual Comparison:

```
Non-Streaming:
[User waits...........................] → Full response appears

Streaming:
[User waits..] → "The" → "answer" → "is" → "4" → "."
```

Let's see streaming in action:

In [ ]:
import time

# Create a new message
messages = [
    {'role': 'user', 'content': 'Write a short poem about AI agents.'}
]

print("Streaming response:")
print("-" * 50)

# Use run() instead of run_nonstream()
for response in bot.run(messages=messages):
    # Each iteration gives us updated messages
    # The last message is the assistant's response
    if response and response[-1]['role'] == 'assistant':
        content = response[-1].get('content', '')
        # Clear previous line and print updated content
        print(f"\r{content}", end='', flush=True)
        time.sleep(0.05)  # Small delay to see streaming effect

print()  # New line after streaming completes
print("-" * 50)

### How Streaming Works:

```python
for response in bot.run(messages):
    # response is a List[Message]
    # Each iteration may have:
    # - Partial text (incomplete thought)
    # - Function calls (tool usage)
    # - Function results (tool outputs)
    # - Complete response (final iteration)
```

The `run()` method returns an **iterator** that yields progressively complete message lists.

## Part 7: Understanding Message Structure

Messages are the core communication unit in Qwen-Agent.

### Basic Message Format:

```python
{
    'role': 'user',      # Who sent this message
    'content': 'Hello'   # What the message says
}
```

### Role Types:
- `'user'` - Input from the human user
- `'assistant'` - Response from the agent/LLM
- `'system'` - Instructions for the agent
- `'function'` - Results from tool execution

Let's explore different message types:

In [ ]:
# Example 1: User message
user_msg = {
    'role': 'user',
    'content': 'What is the capital of France?'
}

# Example 2: System message (gives agent instructions)
system_msg = {
    'role': 'system',
    'content': 'You are a helpful geography tutor. Keep answers brief.'
}

# Example 3: Multi-turn conversation
conversation = [
    {'role': 'user', 'content': 'Hi! What is 10 + 5?'},
    {'role': 'assistant', 'content': 'Hello! 10 + 5 equals 15.'},
    {'role': 'user', 'content': 'What about multiplying them?'},
    # Agent uses context from previous messages to know "them" = 10 and 5
]

print("Message Examples:")
print(f"User: {user_msg}")
print(f"System: {system_msg}")
print(f"\nConversation: {conversation}")

### Testing System Messages:

In [ ]:
# Create agent with system message
pirate_bot = Assistant(
    llm=llm_cfg,
    system_message='You are a friendly pirate. Always respond in pirate speak!'
)

messages = [
    {'role': 'user', 'content': 'What is your name?'}
]

response = pirate_bot.run_nonstream(messages=messages)
print("Pirate Bot Response:")
for msg in response:
    if msg['role'] == 'assistant':
        print(msg['content'])

## Part 8: Multi-Turn Conversations

Real conversations have multiple back-and-forth exchanges. The agent maintains context by keeping message history.

In [ ]:
# Initialize conversation history
messages = []

# Create a simple agent
chat_bot = Assistant(llm=llm_cfg)

# Helper function for cleaner output
def chat(user_input):
    """Send a message and get response"""
    global messages
    
    # Add user message
    messages.append({'role': 'user', 'content': user_input})
    print(f"\n👤 User: {user_input}")
    
    # Get agent response
    response = chat_bot.run_nonstream(messages=messages)
    
    # Display assistant's response
    for msg in response:
        if msg['role'] == 'assistant':
            print(f"🤖 Agent: {msg['content']}")
    
    # Update conversation history
    messages.extend(response)

# Simulate a conversation
print("Starting conversation...")
chat("My name is Alex.")
chat("What's the weather like today?")
chat("What's my name?")  # Tests if agent remembers context

### How Context Works:

```python
messages = []

# Turn 1
messages = [{'role': 'user', 'content': 'I like pizza'}]
response = bot.run_nonstream(messages)
messages.extend(response)  # Now messages has user + assistant messages

# Turn 2
messages.append({'role': 'user', 'content': 'What do I like?'})
# Agent sees full history: [turn1_user, turn1_assistant, turn2_user]
response = bot.run_nonstream(messages)
```

The agent **always** receives the full message history, so it can reference earlier parts of the conversation.

## Part 9: Different LLM Configurations

Let's explore various ways to configure the LLM backend.

In [ ]:
# Configuration 1: Basic (uses defaults)
basic_cfg = {
    'model': 'qwen-max-latest'
}

# Configuration 2: With generation parameters
creative_cfg = {
    'model': 'qwen-max-latest',
    'generate_cfg': {
        'top_p': 0.9,        # Nucleus sampling (higher = more creative)
        'temperature': 1.0,  # Not always supported, check model docs
    }
}

# Configuration 3: With token limits
efficient_cfg = {
    'model': 'qwen-max-latest',
    'generate_cfg': {
        'max_input_tokens': 6000,  # Truncate if input too long
    }
}

# Configuration 4: Different model (smaller/faster)
fast_cfg = {
    'model': 'qwen-turbo-latest',  # Faster, cheaper, less capable
}

print("LLM Configurations:")
print(f"Basic: {basic_cfg}")
print(f"Creative: {creative_cfg}")
print(f"Efficient: {efficient_cfg}")
print(f"Fast: {fast_cfg}")

### Comparing Different Models:

Let's test the same prompt with different models:

In [ ]:
import time

prompt = "In exactly 10 words, explain what an AI agent is."

models_to_test = [
    ('qwen-max-latest', 'Most capable'),
    ('qwen-turbo-latest', 'Faster, cheaper'),
]

for model_name, description in models_to_test:
    print(f"\n{'='*60}")
    print(f"Model: {model_name} ({description})")
    print('='*60)
    
    cfg = {'model': model_name}
    test_bot = Assistant(llm=cfg)
    
    start = time.time()
    response = test_bot.run_nonstream([{'role': 'user', 'content': prompt}])
    elapsed = time.time() - start
    
    for msg in response:
        if msg['role'] == 'assistant':
            print(f"Response: {msg['content']}")
    
    print(f"Time: {elapsed:.2f}s")

## Part 10: Practice Exercises

Now it's your turn! Complete these exercises to solidify your understanding.

### Exercise 1: Create a Math Tutor
Create an agent with a system message that makes it act as a patient math tutor.

In [ ]:
# TODO: Create a math tutor agent
# Requirements:
# 1. System message should instruct it to be a patient math tutor
# 2. It should explain concepts step-by-step
# 3. Test it with a math question

# Your code here:
math_tutor = None  # Replace with your agent

# Test it:
# question = "How do I calculate the area of a circle?"
# ...

### Exercise 2: Implement a Simple Chatbot Loop
Create an interactive chatbot that takes input until the user types 'quit'.

In [ ]:
# TODO: Implement a chatbot loop
# Requirements:
# 1. Maintain conversation history
# 2. Accept user input
# 3. Exit when user types 'quit'
# 4. Print agent responses

# Note: In Jupyter, input() works but might be awkward
# You can test with a predefined list of inputs instead

# Your code here:
def chatbot_loop():
    """Interactive chatbot"""
    # Your implementation
    pass

# Uncomment to test:
# chatbot_loop()

### Exercise 3: Compare Streaming Performance
Measure the time to first token for streaming vs. total time for non-streaming.

In [ ]:
# TODO: Compare streaming vs non-streaming
# Measure:
# 1. Time to first token (streaming)
# 2. Total time to completion (both)
# 3. Print comparison

import time

test_prompt = "Explain quantum computing in simple terms."

# Your code here:
# ...

## Part 11: Key Takeaways

### What You Learned Today:

1. **LLM Agents vs. Direct LLM Usage**
   - Agents add planning, tool use, and memory
   - More powerful for complex tasks

2. **Qwen-Agent Architecture**
   - Agents (orchestration)
   - Tools (capabilities)
   - LLMs (reasoning)
   - Messages (communication)

3. **Environment Setup**
   - Install with pip
   - Configure API keys
   - Verify dependencies

4. **Basic Agent Usage**
   - `Assistant` class for general use
   - `run_nonstream()` for simple cases
   - `run()` for streaming responses

5. **Message Structure**
   - Role + content format
   - Message history for context
   - System messages for instructions

6. **LLM Configuration**
   - Model selection
   - Generation parameters
   - Different backends

### Common Patterns:

```python
# Pattern 1: One-shot question
bot = Assistant(llm={'model': 'qwen-max-latest'})
response = bot.run_nonstream([{'role': 'user', 'content': 'Question?'}])

# Pattern 2: Streaming response
for chunk in bot.run(messages):
    # Process incremental response
    pass

# Pattern 3: Multi-turn conversation
messages = []
messages.append({'role': 'user', 'content': 'Hi'})
response = bot.run_nonstream(messages)
messages.extend(response)
# Continue conversation...
```

## Part 12: Next Steps

### Tomorrow (Day 2): Message Schema Deep Dive
We'll explore:
- The `Message` class in detail
- Multimodal content (text + images)
- ContentItem structure
- Function call messages (preview)
- Building complex message structures

### Homework:
1. Experiment with different system messages
2. Try different Qwen models (qwen-max, qwen-turbo, etc.)
3. Build a simple persona-based chatbot (e.g., "tech support agent")
4. Read the official docs: `/docs/agent.md`

### Resources:
- [Qwen-Agent GitHub](https://github.com/QwenLM/Qwen-Agent)
- [DashScope Models](https://help.aliyun.com/zh/dashscope/)
- [Qwen Documentation](https://qwen.readthedocs.io/)

### Troubleshooting:
- **API key errors**: Check environment variable is set
- **Import errors**: Verify installation with `pip show qwen-agent`
- **Slow responses**: Try `qwen-turbo-latest` for faster replies
- **GUI not working**: Requires Python 3.10+

---

## 🎉 Congratulations!

You've completed Day 1! You now have:
- ✅ A working Qwen-Agent installation
- ✅ Understanding of basic concepts
- ✅ Ability to create and use simple agents
- ✅ Knowledge of streaming vs. non-streaming

See you tomorrow for Day 2! 🚀